# Orientation
Orient the horizontal OBS channels, here using [OrientPy](https://nfsi-canada.github.io/OrientPy/index.html)

The recommended installation is in a dedicated conda environment, which you can create in your shell with the following commands:

```bash
conda create -n orient -c conda-forge python=3.12 obspy
conda activate orient
pip install git+https://github.com/schaefferaj/stdb
pip install geographiclib
pip install git+https://github.com/nfsi-canada/orientpy
```

Most of the commands are run from the shell

## A bug in orientpy
As of 23 July 2025, there is a bug in processing location codes that are more than one letter.  They are aware of it.

If the bug is still there, you will need to modify line 100 in ``io.py`` from:

```python
    for loc in sta.location:
```

to:

```python
    for loc in [sta.location]:
```

## Downloading the stations database
OrientPy accepts StationXML files as input.

Below, we get OBS stations from the RHUM-RUM experiment (YV, 2012-2013)

In [ ]:
from read_data_metadata import read_data, read_metadata

nslc, st, et, server = "YV.RR*.*.*", '2012-11-01', '2013-11-01', "RESIF"

# Creates an inventory file named {DATA}/YV.RRx.xml
inv = read_metadata(server, nslc, st, et)


## Run P-wave polarization

```bash
    conda activate orient
    bng_calc --server-cat 'USGS' --server-wf 'RESIF' --keys RR29,RR34 DATA/YV.RRx.xml
    bng_average --keys RR29,RR34 DATA/YV.RRx.xml
```

My output for the latter was: 

```
###############################################################
#  _                                                          #
# | |__  _ __   __ _     __ ___   _____ _ __ __ _  __ _  ___  #
# | '_ \| '_ \ / _` |   / _` \ \ / / _ \ '__/ _` |/ _` |/ _ \ #
# | |_) | | | | (_| |  | (_| |\ V /  __/ | | (_| | (_| |  __/ #
# |_.__/|_| |_|\__, |___\__,_| \_/ \___|_|  \__,_|\__, |\___| #
#              |___/_____|                        |___/       #
#                                                             #
###############################################################

|==============================================|
|                       RR29                   |
|==============================================|
|  Station: YV.RR29                            |
|      Channel: BH; Locations: 0,0             |
|      Lon:   51.75; Lat: -24.97               |
| Input Directory:  BNG_RESULTS
| Plot Results:  False
|
|    BNG mean, error, data included: 264.4, 7.1, 2

* BNG results are saved in:
*   BNG_RESULTS/YV.RR29/YV.RR29.BNG_results.csv

|==============================================|
|                       RR34                   |
|==============================================|
|  Station: YV.RR34                            |
|      Channel: BH; Locations: 0,0             |
|      Lon:   52.21; Lat: -32.08               |
| Input Directory:  BNG_RESULTS
| Plot Results:  False
|
|    BNG mean, error, data included: 174.1, 91.3, 5

* BNG results are saved in:
*   BNG_RESULTS/YV.RR34/YV.RR34.BNG_results.csv

```

- RR29 has acceptable error, results agree with Scholz et al [2017] (267° +- 7°).
- RR34 has very large error (90°).
    - probably need to look at ``bng_calc`` and ``bng_avg`` plots (``--plot``) and better select the input EQs.

# Run Rayleigh-wave polarization

You can use the same arguments as ``bng_calc``:

```bash
    dl_calc --server-cat 'USGS' --server-wf 'RESIF' --keys RR29,RR34 DATA/YV.RRx.xml
    dl_average --keys RR29,RR34 DATA/YV.RRx.xml
```
 (``dl_calc`` takes a lot longer than ``bng_calc``)

My output for ``dl_average`` was: 

```
#####################################################
#      _ _                                          #
#   __| | |    __ ___   _____ _ __ __ _  __ _  ___  #
#  / _` | |   / _` \ \ / / _ \ '__/ _` |/ _` |/ _ \ #
# | (_| | |  | (_| |\ V /  __/ | | (_| | (_| |  __/ #
#  \__,_|_|___\__,_| \_/ \___|_|  \__,_|\__, |\___| #
#        |_____|                        |___/       #
#                                                   #
#####################################################

|==============================================|
|                       RR29                   |
|==============================================|
|  Station: YV.RR29                            |
|      Channel: BH; Locations: 0,0             |
|      Lon:   51.75; Lat: -24.97               |
| Input Directory:  DL_RESULTS
| Plot Results:  False
|
|    DL mean, error, # robust: 264.91, 2.58, 517
|    DL CC level: 0.80

* DL results are saved in:
*   DL_RESULTS/YV.RR29/YV.RR29.DL_results.csv

|==============================================|
|                       RR34                   |
|==============================================|
|  Station: YV.RR34                            |
|      Channel: BH; Locations: 0,0             |
|      Lon:   52.21; Lat: -32.08               |
| Input Directory:  DL_RESULTS
| Plot Results:  False
|
|    DL mean, error, # robust: 131.07, 2.85, 464
|    DL CC level: 0.80

* DL results are saved in:
*   DL_RESULTS/YV.RR34/YV.RR34.DL_results.csv
```
- Many more acceptable orientations (used ``--plot`` to see) than ``bng*``
- RR28 has small error, still agrees with Scholz et al.
- RR34 has small error, agrees with Scholz et al (131° += 4°)


A401A had by far the cleanest looking data, A419A had the best low-frequency coherency

## Exercises

- Compare the results with those of [Scholz et al. 2017](https://doi.org/10.1093/gji/ggw426) and [Trabattoni et al. 2020](https://doi.org/10.1093/gji/ggz519)